In [1]:
import tensorflow as tf
import numpy as np
from astropy.io import fits
from vacuum.io_ import load_data, preprocess, deprocess, fits_open
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
INPUT_DIR = "/scratch/datasets/astro_deconv_2019"
batch, count = load_data(INPUT_DIR)
iter = batch.make_one_shot_iterator()
index, min_flux, max_flux, psf, dirty, skymodel = iter.get_next()
IM = skymodel
ID = dirty
PSF = psf

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [3]:
ID.shape

TensorShape([Dimension(None), Dimension(256), Dimension(256), Dimension(1)])

In [20]:
_, npix, _, _ = ID.shape
IMhat = tf.signal.rfft2d(IM)
PSFhat = tf.signal.rfft2d(PSF)
mult = tf.multiply(IMhat,PSFhat)[0, :, :, 0]
PSFconvI = tf.signal.irfft2d(mult)
A = tf.squeeze(IM)
B = tf.squeeze(PSFconvI) - 2 * tf.squeeze(ID)
loss_tf = tf.reduce_sum(tf.multiply(A, B))

In [21]:
with tf.Session() as sess:
  dang = sess.run(B)

#_ = plt.imshow(dang)

InvalidArgumentError: Incompatible shapes: [256,510] vs. [256,256]
	 [[node sub_4 (defined at <ipython-input-20-1401bad3fb38>:7) ]]
	 [[node sub_4 (defined at <ipython-input-20-1401bad3fb38>:7) ]]

Caused by op 'sub_4', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/gijs/Work/vacuum-cleaner/.venv3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/gijs/Work/vacuum-cleaner/.venv3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/gijs/Work/vacuum-cleaner/.venv3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/gijs/Work/vacuum-cleaner/.venv3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asynci

In [24]:
PSFconvI.shape

TensorShape([Dimension(256), Dimension(510)])